# Colabで全文検索（その７：RediSearch編）

各種全文検索ツールをColabで動かしてみるシリーズです。今回は[Redis](https://redis.io/)の拡張モジュール[RediSearch](https://oss.redis.com/redisearch/)です。

処理時間の計測はストレージのキャッシュとの兼ね合いがあるので、2回測ります。2回目は全てがメモリに載った状態での性能評価になります。ただ1回目もデータを投入した直後なので、メモリに載ってしまっている可能性があります。

## 準備

まずは検索対象のテキストを日本語wikiから取得して、Google Driveに保存します。

Google Driveのマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


jawikiの取得とjson形式に変換。90分ほど時間がかかります。

In [2]:
%%time
%cd /content/
import os
if not os.path.exists('/content/drive/MyDrive/wiki.json.bz2'):
    !wget https://dumps.wikimedia.org/jawiki/latest/jawiki-latest-pages-articles.xml.bz2
    !pip install wikiextractor
    !python -m wikiextractor.WikiExtractor --no-templates --processes 4 --json -b 10G -o - jawiki-latest-pages-articles.xml.bz2 | bzip2 -c > /content/drive/MyDrive/wiki.json.bz2

/content
CPU times: user 3.41 ms, sys: 50 µs, total: 3.46 ms
Wall time: 8.8 ms


json形式に変換されたデータを確認

In [3]:
import json
import bz2

with bz2.open('/content/drive/MyDrive/wiki.json.bz2', 'rt', encoding='utf-8') as fin:
    for n, line in enumerate(fin):
        data = json.loads(line)
        print(data['title'].strip(), data['text'].replace('\n', '')[:40], sep='\t')
        if n == 5:
            break

アンパサンド	アンパサンド（&amp;, ）は、並立助詞「…と…」を意味する記号である。ラテン
言語	言語（げんご）は、広辞苑や大辞泉には次のように解説されている。『日本大百科事典』
日本語	 日本語（にほんご、にっぽんご）は、日本国内や、かつての日本領だった国、そして日
地理学	地理学（ちりがく、、、伊：geografia、）は、。地域や空間、場所、自然環境
EU (曖昧さ回避)	EU
国の一覧	国の一覧（くにのいちらん）は、世界の独立国の一覧。対象.国際法上国家と言えるか否


## Redisのインストール

Redisを取得して、インストールします。

In [4]:
!sudo yes | add-apt-repository ppa:redislabs/redis
!sudo apt-get update
!sudo apt-get install redis

 Redis is an open source (BSD licensed), in-memory data structure store, used as a database, cache and message broker.

It supports data structures such as strings, hashes, lists, sets, sorted sets with range queries, bitmaps, hyperloglogs, geospatial indexes with radius queries and streams.

Redis has built-in replication, Lua scripting, LRU eviction, transactions and different levels of on-disk persistence, and provides high availability via Redis Sentinel and automatic partitioning with Redis Cluster.
 More info: https://launchpad.net/~redislabs/+archive/ubuntu/redis
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu

インストールされたRedisのバージョンを確認します。

In [5]:
!redis-cli --version
!redis-server --version

redis-cli 7.0.0
Redis server v=7.0.0 sha=00000000:0 malloc=jemalloc-5.2.1 bits=64 build=698b5df4772f6164


In [6]:
!!echo "enable-module-command yes" >> /etc/redis/redis.conf

[]

Redisを起動します。

In [7]:
!service redis-server start

Starting redis-server: redis-server.


## RediSearchのビルド

Redisの拡張モジュールであるRediSearchを取得してビルドします。

In [8]:
%cd /content
!git clone --recursive https://github.com/RediSearch/RediSearch.git
%cd /content/RediSearch
!make setup
!make build

/content
Cloning into 'RediSearch'...
remote: Enumerating objects: 39420, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 39420 (delta 93), reused 66 (delta 40), pack-reused 39256
Receiving objects: 100% (39420/39420), 26.49 MiB | 20.31 MiB/s, done.
Resolving deltas: 100% (29459/29459), done.
Submodule 'deps/VectorSimilarity' (https://github.com/RedisAI/VectorSimilarity.git) registered for path 'deps/VectorSimilarity'
Submodule 'deps/googletest' (https://github.com/google/googletest.git) registered for path 'deps/googletest'
Submodule 'deps/hiredis' (https://github.com/redis/hiredis.git) registered for path 'deps/hiredis'
Submodule 'deps/libuv' (https://github.com/libuv/libuv.git) registered for path 'deps/libuv'
Submodule 'deps/readies' (https://github.com/RedisLabsModules/readies.git) registered for path 'deps/readies'
Cloning into '/content/RediSearch/deps/VectorSimilarity'...
remote: Enumerating objects: 2658, d

redisearch.soの場所を確認して、RediSearchモジュールをロードします。

In [9]:
!find /content/RediSearch -name redisearch.so

/content/RediSearch/bin/linux-x64-release/search/redisearch.so


In [10]:
!redis-cli module load /content/RediSearch/bin/linux-x64-release/search/redisearch.so

OK


## データのインポート

Pythonのredisearchモジュールをインストールします。

In [11]:
!pip install redisearch

     |████████████████████████████████| 85 kB 2.7 MB/s 
     |████████████████████████████████| 72 kB 425 kB/s 
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: redis
    Found existing installation: redis 4.2.2
    Uninstalling redis-4.2.2:
      Successfully uninstalled redis-4.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
rltest 0.5.4 requires redis<5.0.0,>=4.1.2, but you have redis 3.5.3 which is incompatible.
ramp-packer 2.3.0 requires redis<5.0.0,>=4.1.0, but you have redis 3.5.3 which is incompatible.
google-colab 1.0.0 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
datascience 0.10.6 requires coverage==3.7.1, but you h

50万件のデータをインポートします。RediSearchには日本語を分割する機能がないので、一文字づつスペースで区切った形でインポートします。

In [12]:
from redisearch import Client, TextField, NumericField, Query
client = Client('myIndex')
client.create_index([TextField('body')])

'OK'

In [13]:
%%time 

import json
import bz2
from tqdm.notebook import tqdm

limit = 500000
with bz2.open('/content/drive/MyDrive/wiki.json.bz2', 'rt', encoding='utf-8') as fin:
    n = 0
    for line in tqdm(fin, total=limit*1.5):
        data = json.loads(line)
        title = data['title'].strip()
        body = data['text'].replace('\n', '')
        if len(title) > 0 and len(body) > 0:
            client.add_document("doc_{}".format(n), title=title, body=' '.join(body))
            n += 1
        if n == limit:
            break


  0%|          | 0/750000.0 [00:00<?, ?it/s]

CPU times: user 8min 45s, sys: 40.9 s, total: 9min 26s
Wall time: 16min 40s


## 検索

1回目の検索。検索文字列は"日 本 語"のようにスペースで区切って指定します。in_order()は"日","本","語"を順番通りに並ばせる指定、slop(0)はこれらの文字の間になにも入らない指定、paging(0, 20000)は2万件以下を一気に検索する指定です。

In [14]:
%%time
query = Query("日 本 語").in_order().slop(0).paging(0, 20000)
res = client.search(query)
print('total:', res.total, 'time:', '{:.2f} sec'.format(res.duration / 1000))

total: 17007 time: 1.25 sec
CPU times: user 770 ms, sys: 216 ms, total: 986 ms
Wall time: 1.41 s


検索結果数も他と合っているので、正しく検索できているようです。

2回目の検索。

In [15]:
%%time
query = Query("日 本 語").in_order().slop(0).paging(0, 20000)
res = client.search(query)
print('total:', res.total, 'time:', '{:.2f} sec'.format(res.duration / 1000))

total: 17007 time: 1.17 sec
CPU times: user 655 ms, sys: 209 ms, total: 864 ms
Wall time: 1.27 s


検索結果を確認します。

In [16]:
res.docs[1]

Document {'id': 'doc_226353', 'payload': None, 'body': 'エ イ ジ 、 エ ー ジ 日 本 語 . 人 名 . 日 本 語 の 男 性 名 で 用 い ら れ る 。', 'title': 'エイジ'}

検索対象をbodyフィールドのみにして検索。

In [17]:
%%time
query = Query('@body:"日 本 語"').in_order().slop(0).paging(0, 20000)
res = client.search(query)
print('total:', res.total, 'time:', '{:.2f} sec'.format(res.duration / 1000))

total: 17007 time: 1.15 sec
CPU times: user 856 ms, sys: 152 ms, total: 1.01 s
Wall time: 1.41 s


あまり変わりません。

次に、検索結果のコンテンツを取得せず、idだけを取得する検索。

In [18]:
%%time
query = Query('@body:"日 本 語"').in_order().slop(0).no_content().paging(0, 20000)
res = client.search(query)
print('total:', res.total, 'time:', '{:.2f} sec'.format(res.duration / 1000))

total: 17007 time: 0.23 sec
CPU times: user 92.3 ms, sys: 46.1 ms, total: 138 ms
Wall time: 325 ms


非常に速い。

検索結果を確認します。idのみが取れているのが確認できます。

In [19]:
res.docs[1]

Document {'id': 'doc_226353', 'payload': None}

## Redisの停止

In [20]:
!service redis-server stop

Stopping redis-server: redis-server.
